In [10]:
import xarray as xr
import numpy as np

ds = xr.open_dataset("/Users/rem76/Desktop/Climate_change_health/Data/Temperature_data/Historical/Hourly/7ba7ebaaf13bddcb13b9d30aaa6f1512.nc")  # contains t2m (K) and d2m (K)
ds_weekly = ds.resample(valid_time="7D").mean()
# 
# 
# 
# Tc  = ds["t2m"] - 273.15
# Tdc = ds["d2m"] - 273.15
# 
# es = 6.112 * np.exp(17.625 * Tc  / (243.04 + Tc))   # hPa
# e  = 6.112 * np.exp(17.625 * Tdc / (243.04 + Tdc))  # hPa
# 
# rh = 100.0 * (e / es)
# rh = rh.clip(0, 100)
# 
# rh = rh.rename("rh2m")
# rh.attrs.update(long_name="2 m relative humidity", units="%")


In [12]:
ds_weekly

<xarray.Dataset> Size: 1GB
Dimensions:     (valid_time: 22, latitude: 1801, longitude: 3600)
Coordinates:
    number      int64 8B 0
  * latitude    (latitude) float64 14kB 90.0 89.9 89.8 ... -89.8 -89.9 -90.0
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
  * valid_time  (valid_time) datetime64[ns] 176B 2019-01-01 ... 2019-05-28
Data variables:
    d2m         (valid_time, latitude, longitude) float32 571MB nan ... 215.5
    t2m         (valid_time, latitude, longitude) float32 571MB nan ... 219.4
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-20T09:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [13]:
import pandas as pd

# Load data
mod_b = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_B.csv")
mod_e = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_E.csv")
geo = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/householdgeovariables_ihs5.csv")

# Merge B and E
merged = pd.merge(mod_b, mod_e, on=["case_id", "HHID", "PID"], how="inner")

# Merge with geo on case_id and HHID
merged_geo = pd.merge(merged, geo, on=["case_id", "case_id"], how="left")

# Subset: rows containing HEALTH WORKER, DOCTOR, NURSE in any column
keywords = ["HEALTH WORKER", "DOCTOR", "NURSE"]
mask_keywords = merged_geo.apply(
    lambda row: row.astype(str).str.contains('|'.join(keywords), case=False, na=False).any(), axis=1
)
subset_health = merged_geo[mask_keywords]

# Sum of hours worked
subset_health["hh_e11"] = pd.to_numeric(subset_health["hh_e11"], errors="coerce")
sum_hours_health = subset_health["hh_e11"].sum()

# Rows that have BAD WEATHER or NATURAL DISASTER
subset_weather = merged_geo[merged_geo["hh_e15"].str.contains("BAD WEATHER|NATURAL DISASTER", case=False, na=False)]

print("Sum of hh_e11 for HEALTH WORKER/DOCTOR/NURSE subset:", sum_hours_health)
print("Subset with BAD WEATHER or NATURAL DISASTER in hh_e15:")
#print(subset_weather.head())

/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_4482/1796722484.py:4: DtypeWarning: Columns (17,25,35,37,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  mod_b = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_B.csv")
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_4482/1796722484.py:5: DtypeWarning: Columns (38,42,44,45,46,64,88,90,92,93,94,95,105,117,122,125,126,133,158) have mixed types. Specify dtype option on import or set low_memory=False.
  mod_e = pd.read_csv("/Users/rem76/Desktop/Climate_change_health/World_Bank_Household_Survey_Data/MWI_2019_IHS-V_v06_M_CSV/HH_MOD_E.csv")


Sum of hh_e11 for HEALTH WORKER/DOCTOR/NURSE subset: 1597.0
Subset with BAD WEATHER or NATURAL DISASTER in hh_e15:


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_4482/1796722484.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health["hh_e11"] = pd.to_numeric(subset_health["hh_e11"], errors="coerce")


In [16]:
subset_health

,case_id,HHID,PID,hh_b03,hh_b04,hh_b04a,hh_b05_2,hh_b05_2_1,hh_b05a,hh_b05b,...,wetQ_avgstart,wetQ_avg,h2018_ndvi_avg,h2018_ndvi_max,h2019_ndvi_avg,h2019_ndvi_max,ndvi_avg,ndvi_max,ea_lat_mod,ea_lon_mod
2765,102041370093,7fbd200d6dc54ddea90a6597c7686557,1,MALE,HEAD,YES,NaN,NaN,41,NaN,...,18.0,711.0,0.231,0.524,0.181,0.345,0.229,0.456,-10.44482,34.106800
3365,102201450188,76bbaf58d54f4c91b63512d6b77d0bfc,2,FEMALE,WIFE/HUSBAND,YES,NaN,NaN,27,NaN,...,19.0,804.0,0.081,0.268,0.084,0.199,0.104,0.297,-9.94791,33.906231
3373,102201450252,1411371af23b4a598e488d4822467984,1,MALE,HEAD,YES,NaN,NaN,22,NaN,...,19.0,804.0,0.081,0.268,0.084,0.199,0.104,0.297,-9.94791,33.906231
3845,103021520071,e4e7e239c17d4ca5b4baf36302380c84,2,FEMALE,WIFE/HUSBAND,NO,NaN,NaN,37,NaN,...,18.0,767.0,0.100,0.360,0.084,0.315,0.037,0.301,-11.96827,34.080139
6108,104071820041,696daf3ac40043a0aa20f5a416fbbc9b,1,MALE,HEAD,YES,NaN,NaN,27,NaN,...,18.0,711.0,0.300,0.575,0.202,0.373,0.273,0.485,-10.92589,34.070820
8523,107362180099,d516c1cfb2ce4c838259ba60226e511a,2,FEMALE,WIFE/HUSBAND,YES,NaN,NaN,37,NaN,...,18.0,719.0,0.328,0.546,0.226,0.448,0.284,0.496,-11.45438,34.027401
8547,107362180149,ab5e728f068e42359a04448f2231c901,1,MALE,HEAD,YES,NaN,NaN,30,NaN,...,18.0,719.0,0.328,0.546,0.226,0.448,0.284,0.496,-11.45438,34.027401
8720,107372210076,0e1fdf7b916546f7994528971a7ed1fa,2,FEMALE,WIFE/HUSBAND,YES,NaN,NaN,39,NaN,...,18.0,696.0,0.302,0.550,0.203,0.397,0.281,0.489,-11.40003,34.008121
8983,107392250204,cfe1060884514a03a665e69d32d12dea,3,MALE,CHILD/ADOPTED CHILD,YES,NaN,NaN,23,NaN,...,18.0,707.0,0.296,0.519,0.194,0.401,0.273,0.477,-11.44342,33.996288
9067,107392260141,3593b5cc0f0a42f4bdec98b89a0ed244,1,MALE,HEAD,YES,NaN,NaN,35,NaN,...,18.0,712.0,0.303,0.526,0.197,0.412,0.274,0.481,-11.45704,33.997608


In [24]:

keywords = ["HEALTH WORKER", "DOCTOR", "NURSE"]
pattern = "|".join(keywords)  # "HEALTH WORKER|DOCTOR|NURSE"

subset = mod_e[mod_e["hh_e19a"].str.contains(pattern, case=False, na=False)]
subset

,case_id,HHID,PID,hh_e03,hh_e04,hh_e05,hh_e06,hh_e06_1a,hh_e06_1b,hh_e06_1c,...,hh_e63,hh_e64,hh_e65,hh_e66,hh_e67,hh_e68,hh_e69,hh_e70,hh_e71,hh_e71_oth
2765,102041370093,7fbd200d6dc54ddea90a6597c7686557,1,NO,2.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
3365,102201450188,76bbaf58d54f4c91b63512d6b77d0bfc,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
6108,104071820041,696daf3ac40043a0aa20f5a416fbbc9b,1,YES,NaN,0.2,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,Yes,Yes,3.0,No,NaN,NaN
8523,107362180099,d516c1cfb2ce4c838259ba60226e511a,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8547,107362180149,ab5e728f068e42359a04448f2231c901,1,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8720,107372210076,0e1fdf7b916546f7994528971a7ed1fa,2,YES,NaN,0.0,0.0,YES,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
8983,107392250204,cfe1060884514a03a665e69d32d12dea,3,NO,1.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,Yes,Yes,Yes,4.0,Yes,To have a better paid job,NaN
9067,107392260141,3593b5cc0f0a42f4bdec98b89a0ed244,1,YES,NaN,0.0,0.0,YES,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
9074,107392260165,e3200be4c72042fe85d5189d7d6c954c,2,YES,NaN,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN
9294,107402300015,df216a6d39cb42f4a996bc526f55b381,1,NO,2.0,0.0,0.0,NO,NO,NO,...,NaN,NaN,NaN,No,No,NaN,NaN,No,NaN,NaN
